# <b>Task:</b> Implementation of a simulated annealing method. The algorithm is implemented for the Cell formation problem

The data for the work is uploaded to the repositories.

Structure:

m p (number of machines and parts)

Next m rows:

m(row number) list of parts processed by machine m separate by space 

##Подключение библиотек, чтение данных, вспомогательные функции

In [ ]:
import random
import numpy as np
import time
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/data_lab5.zip'


Archive:  /content/drive/MyDrive/data_lab5.zip
   creating: cfp_data/
  inflating: cfp_data/20x20.txt      
  inflating: cfp_data/24x40.txt      
  inflating: cfp_data/30x50.txt      
  inflating: cfp_data/30x90.txt      
  inflating: cfp_data/37x53.txt      


In [ ]:
def check_time(f, *args):
  function_time = 0.
  for i in np.arange(1):
    start = time.time()
    f(*args)
    function_time += time.time() - start
  avarege_time = function_time / 1
  return avarege_time

In [ ]:
def read_data_for_test(file_path):
  data = []
  with open(file_path) as f:
    for line in f:
        data.append([int(x) for x in line.split()])
  size = [data[0][0], data[0][1]]
  matrix = np.zeros([data[0][0], data[0][1]])
  for i in range(1,len(data)):
    for j in range(1,len(data[i])):
      matrix[data[i][0]-1][data[i][j]-1] = 1
  return size, matrix



In [ ]:
def print_data_for_test(size, matrix):
  print("ДАННЫЕ:\n")
  print("Количество машин и деталей: ", size)
  print("\nМатрица  ======================================================\n\n")
  for i in range(size[0]):
    for j in range(size[1]):
      print(int(matrix[i][j]),end=' ')
    print('\n')


In [ ]:
 #reading test
size, matrix = read_data_for_test('cfp_data/20x20.txt')
print_data_for_test(size, matrix)
size, matrix = read_data_for_test('cfp_data/24x40.txt')
print_data_for_test(size, matrix)
size, matrix = read_data_for_test('cfp_data/30x50.txt')
print_data_for_test(size, matrix)
size, matrix = read_data_for_test('cfp_data/30x90.txt')
print_data_for_test(size, matrix)
size, matrix = read_data_for_test('cfp_data/37x53.txt')
print_data_for_test(size, matrix)


ДАННЫЕ:

Количество машин и деталей:  [20, 20]

Матрица  ======================================================


1 0 0 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 0 0 

0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 1 0 0 1 

0 0 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 

0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 

0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 

0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 

1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 1 1 

0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 0 1 0 1 

0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 

0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 

0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 

0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 

0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0 

0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 

0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 

0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 

1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 

0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 

1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 

ДАННЫЕ:

Количество машин и деталей:  [24, 40]

# Simulated annealing method

In [ ]:
import math
import random
import numpy as np

def similarity_matrix_for_parts(i, j, size, matrix):
    a_ij = 0
    b_ij = 0
    c_ij = 0
    for x in np.arange(size[0]):
        if matrix[x][i] == 1 and matrix[x][j] == 1:
            a_ij += 1
        elif matrix[x][i] == 1:
            b_ij += 1
        elif matrix[x][j] == 1:
            c_ij += 1
    return a_ij / (a_ij + b_ij + c_ij)


def objective_function_value(cells_machines, cells_details, matrix, num_of_cells):
    n_1_in = 0
    n_1 = 0
    n_0_in = 0
    n_1_out = 0
    for x in matrix:
        for y in x:
            n_1_out += y
            n_1 += y
    for i in np.arange(num_of_cells):
        for machines in cells_machines[i]:
            for detail in cells_details[i]:
                n_1_in += matrix[machines][detail]
                if matrix[machines][detail] == 0:
                    n_0_in += 1
    n_1_out -= n_1_in
    return (n_1 - n_1_out) / (n_1 + n_0_in)


def single_move(size, current_solve, matrix, num_of_cells):
    cells_details = current_solve[1].copy()
    best_f = -9999
    best_cells_details = []
    best_cells_machine = []
    detail_to_move = random.randint(0, size[1] - 1)
    old_cell = 0
    for i in np.arange(len(cells_details)):
        if detail_to_move in cells_details[i]:
            old_cell = i
            cells_details[old_cell].remove(detail_to_move)
            break
    for i in np.arange(len(cells_details)):
        if i != old_cell:
            cells_details[i].append(detail_to_move)
            new_cells_machine = machines_assignment(matrix, cells_details, num_of_cells)
            new_f = objective_function_value(new_cells_machine, cells_details, matrix, num_of_cells)
            if new_f > best_f:
                best_cells_machine = new_cells_machine.copy()
                best_cells_details = cells_details.copy()
                best_f = new_f
            cells_details[i].remove(detail_to_move)
    cells_details[old_cell].append(detail_to_move)
    return best_f, best_cells_details, best_cells_machine


def exchange_move(size, current_solve, matrix, num_of_cells):
    cells_details = current_solve[1].copy()
    best_f = -9999
    best_cells_details = []
    best_cells_machine = []
    detail_to_move = random.randint(0, size[1] - 1)
    old_cell = 0
    for i in np.arange(len(cells_details)):
        if detail_to_move in cells_details[i]:
            old_cell = i
            cells_details[old_cell].remove(detail_to_move)
            break
    for i in np.arange(len(cells_details)):
        if i != old_cell and len(cells_details[i]) != 0:
            detail_to_move2 = cells_details[i][random.randint(0, len(cells_details[i]) - 1)]
            cells_details[i].append(detail_to_move)
            cells_details[i].remove(detail_to_move2)
            cells_details[old_cell].append(detail_to_move2)
            new_cells_machine = machines_assignment(matrix, cells_details, num_of_cells)
            new_f = objective_function_value(new_cells_machine, cells_details, matrix, num_of_cells)
            if new_f > best_f:
                best_cells_machine = new_cells_machine.copy()
                best_cells_details = cells_details.copy()
                best_f = new_f
            cells_details[i].remove(detail_to_move)
            cells_details[i].append(detail_to_move2)
            cells_details[old_cell].remove(detail_to_move2)
    cells_details[old_cell].append(detail_to_move)
    return best_f, best_cells_details, best_cells_machine


def machines_assignment(matrix, cells_details, num_of_cells):
    cells_machines = [[] for x in range(num_of_cells)]
    for machine in np.arange(size[0]):
        error = [0 for x in np.arange(num_of_cells)]
        for cell in np.arange(num_of_cells):
            for detail in cells_details[cell]:
                if matrix[machine][detail] == 0:
                    error[cell] += 1
                else:
                    for i in np.arange(num_of_cells):
                        if i != cell:
                            error[i] += 1
        #print(error)
        ans = np.argmin(error)
        #print(ans)
        cells_machines[ans].append(machine)
        #print(cells_machines)

    return cells_machines


def Simulated_annealing_approach(size, matrix, i):
    num_of_cells = 2
    b = 0
    T0 = [10, 10, 8, 7, 10]
    TF = [0.001, 0.001, 0.001, 0.001, 0.001]
    alphai = [0.70, 0.75, 0.8, 0.85, 0.95]
    Li = [10, 30, 500, 1000, 1500]
    Di = [2, 20, 50, 100, 150]
    checki = [100, 100, 100, 150, 200]
    similarity = []

    for i in np.arange(size[1]):
        for j in np.arange(size[1]):
          if i < j:
            b = similarity_matrix_for_parts(i, j, size, matrix)
            if b != 0:
                similarity.append((b, i, j))
    similarity.sort(reverse=True)

    best_solve = [-9999, [], []]

    Flag = True
    while True:
        if Flag:
            delta = [similarity[(x+1)*int(math.sqrt(len(similarity)))][0] for x in np.arange(num_of_cells - 1)]
            cells_details = [[] for x in np.arange(num_of_cells)]
            cells_details[0].append(similarity[0][1])
            cells_details[0].append(similarity[0][2])
            delta_counter = 0
            for i in np.arange(1, len(similarity)):
                if similarity[i][0] > delta[delta_counter]:
                    if similarity[i][1] not in cells_details[delta_counter]:
                        cells_details[delta_counter].append(similarity[i][1])
                    if similarity[i][2] not in cells_details[delta_counter]:
                        cells_details[delta_counter].append(similarity[i][2])
                else:
                    delta_counter += 1
                if delta_counter == num_of_cells - 1:
                    break
            for detail in np.arange(size[1]):
                f = True
                for i in np.arange(num_of_cells-1):
                    if detail in cells_details[i]:
                        f = False
                if f:
                    cells_details[num_of_cells - 1].append(detail)
            cells_machines = machines_assignment(matrix, cells_details, num_of_cells)

            current_best_solve = (objective_function_value(cells_machines, cells_details, matrix, num_of_cells), cells_details, cells_machines)  # S**
            if current_best_solve[0] > best_solve[0]:
                best_solve = current_best_solve  # S*
            current_solve = current_best_solve  # S

            T_0 = T0[i]
            T_f = TF[i]
            T = T_0
            alpha = alphai[i]
            L = Li[i]
            D = Di[i]
            check = checki[i]
            counter = 0
            counter_mc = 0
            counter_trapped = 0
            counter_stagnant = 0

        Flag = False
        while counter_mc < L and counter_trapped < L / 2:
            new_solve = single_move(size, current_solve, matrix, num_of_cells)  # S_c
            if counter % D == 0:
                new_solve = exchange_move(size, current_solve, matrix, num_of_cells)
            if new_solve[0] > current_best_solve[0]:
                current_best_solve = new_solve
                counter_stagnant = 0
                counter_mc += 1
                current_solve = new_solve
                continue
            if new_solve[0] == current_best_solve[0]:
                current_solve = new_solve
                counter_stagnant += 1
                counter_mc += 1
                continue
            delta = new_solve[0] - current_solve[0]
            if math.e ** (delta / T) > random.uniform(0, 1):
                current_solve = new_solve
                counter_trapped = 0
            else:
                counter_trapped += 1
            counter_mc += 1
        if T <= T_f or counter_stagnant > check:
            if current_best_solve[0] > best_solve[0]:
                best_solve = current_best_solve
                num_of_cells += 1
                Flag = True
            else:
                break
        else:
            T = T * alpha
            counter_mc = 0
            counter += 1
            continue
    return best_solve

In [ ]:
files_name = ['cfp_data/20x20.txt','cfp_data/24x40.txt','cfp_data/30x50.txt','cfp_data/30x90.txt','cfp_data/37x53.txt']
#files_name = ['cfp_data/24x40.txt']
i = 0
for file_name in files_name:
  size, matrix  = read_data_for_test(file_name)
  answer = Simulated_annealing_approach(size, matrix, i)
  print("Тест: ", file_name )
  print("Ответ: ", answer)
  print("Время работы: ", check_time(Simulated_annealing_approach, size, matrix, i), "\n")
  i += 1

Тест:  cfp_data/20x20.txt
Ответ:  (0.3235294117647059, [[4, 7, 2, 17, 1, 3, 13, 14, 8, 16, 5, 10, 12], [15, 18, 19, 9, 6, 11, 0]], [[1, 2, 4, 7, 10, 11, 12, 13, 14, 15, 16], [0, 3, 5, 6, 8, 9, 17, 18, 19]])
Время работы:  2.200162410736084 

Тест:  cfp_data/24x40.txt
Ответ:  (0.23785166240409208, [[19, 6, 24, 17, 28, 22, 16, 1, 30, 3, 23, 14, 18], [36, 29, 35, 12, 26, 24, 17, 31, 25, 1, 37, 3, 13, 7, 4, 10, 39], [11, 38, 0, 27, 21, 9, 33, 20, 32, 5, 15, 8, 2, 34]], [[0, 2, 9, 11, 16, 19], [5, 6, 7, 13, 14, 15, 17, 18, 23], [1, 3, 4, 8, 10, 12, 20, 21, 22]])
Время работы:  28.782238006591797 

Тест:  cfp_data/30x50.txt
Ответ:  (0.1801526717557252, [[25, 0, 26, 6, 5, 3, 18, 12, 29, 10, 22, 19, 1, 17, 8, 2, 23, 9, 7, 4, 14, 24, 27], [25, 26, 49, 6, 5, 3, 15, 10, 44, 1, 17, 11, 23, 4, 32, 13, 24, 27, 39], [38, 33, 41, 34, 46, 47, 21, 40, 28, 48, 37, 35, 36, 45, 30, 42, 16, 43, 20, 31]], [[10, 11, 12, 21, 28], [4, 7, 14, 16, 17, 19, 20, 24, 27], [0, 1, 2, 3, 5, 6, 8, 9, 13, 15, 18, 22, 23, 

IndexError: ignored